Кодирование –выбор «генетического кода» Особь –битовая последовательность размера n (кол-во грузов)

1.Начальная популяция –кол-во особей всегда  = 200:

	1.1 жадный выбор, начиная со случайного груза

2.Отбор особей для скрещивания:

	2.1 выбор каждой особи пропорционально приспособленности (рулетка)


3.Скрещивание (кроссинговер) между выбранными особями.
Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка:

	3.2 однородный (каждый бит от случайно выбранного родителя)

4.Мутация:

	4.1 инвертирование всех битов у 1 особи

5.Формирование новой популяции (кол-во особей -константа)

	5.3 замена своих родителей
    
6.Оценка результата Наступила сходимость (функция приспособленности лучшей особи в популяциях отличается не более,
чем на стоимость самой дешевой вещи) или прошло 500 поколений

In [1]:
import numpy as np
import pandas as pd
import random
pd.options.mode.chained_assignment = None

In [2]:
dataFrame = pd.read_csv('22.txt', sep=" ", header=None, skiprows=1, names=['w', 'v', 'p'])
WVVals = pd.read_csv('22.txt', sep=" ", header=None, nrows=1)

maxWeight = WVVals.iloc[0][0]
maxVolume = WVVals.iloc[0][1]

weightList = dataFrame['w'].values.tolist()
volumeList = dataFrame['v'].values.tolist()
priceList = dataFrame['p'].values.tolist()

In [3]:
def fitness(currSet):
    currWeight = sum([a * b for a, b in zip(weightList, currSet)])
    currVolume = sum([a * b for a, b in zip(volumeList, currSet)])
    currPrice = sum([a * b for a, b in zip(priceList, currSet)])

    if currVolume > maxVolume or currWeight > maxWeight:
        return 0;

    return currPrice;


In [197]:
# Шаг 1 . Случайная генерация
def generateEntity(length):
    newSet = [0 for i in range(length)]
    firstIDX = random.randint(0,length-1)
    newSet[firstIDX] = 1
    idx = firstIDX
    while(True):
        if(idx == length):
            idx = 0
        if(newSet[idx]!=1):
            newSet[idx] = 1
            if(fitness(newSet) == 0):
                newSet[idx] = 0
                break
        idx=idx+1  
    return newSet

def generation(length, entities):
    sets = []
    setID=0;
    for i in range(entities):
        newSet = generateEntity(length)
        sets.append([setID, newSet, fitness(newSet)])
        setID=setID+1
    return pd.DataFrame(sets,columns=['id', 'set', 'fitness'])

In [5]:
# Шаг 2. Массовая игра в русскую рулетку
def sectioning(setsDF):
    setsData = setsDF
    setsData = setsData[setsData['fitness'] != 0]
    totalFitness = sum(setsData['fitness'].values.tolist())
    maxSectVal = 1;

    setsData['section'] = setsData.apply(lambda row: row['fitness']/totalFitness, axis=1)
    setsData = setsData.sort_values(by=['fitness'],ascending=False)

    sectionsFixed = setsData['section'].values.tolist()
    newSections = []
    for fit in sectionsFixed:
        maxSectVal = maxSectVal-fit
        if(maxSectVal<0):
            maxSectVal = 0
        fit = maxSectVal
        newSections.append(maxSectVal)
    setsData['section'] = newSections
    return setsData

def roulette(sectDF):
    while True:
        rand = random.random()
        chosen = sectDF[sectDF['section']>rand]
        if(len(chosen.values.tolist())>0):break
        if (len(sectDF.values.tolist()) == 1):return sectDF.iloc[0]   
    return chosen.iloc[-1]

In [6]:
#Создание пар
def createPairs(setsDF):
    setsDF = sectioning(setsDF)
    setsDF['pair'] = [0 for i in range(len(setsDF['set'].values.tolist()))]
    currPairNum = 0
    while True:
        pairless = setsDF[setsDF['pair']==0]
        pairlessCount = len(pairless.values.tolist())
        pairless = sectioning(pairless)
        if(pairlessCount < 2):
            break
        
        firstParentID = roulette(pairless)['id']
        secondParentID = roulette(pairless[pairless['id'] != firstParentID])['id']
        setsDF.loc[setsDF['id'].isin([firstParentID,secondParentID]),'pair'] = currPairNum
        currPairNum=currPairNum+1
    return setsDF

#Шаг 5. Формирование новой популяции (замена своих родителей)

#Шаг 6. Оценка

In [7]:
#Шаг 3. Кроссинговер (однородный (каждый бит от случайно выбранного родителя))
def breed(setPair):
    return [setPair[random.randint(0,1)][i] for i in range(0,len(setPair[0]))]

In [8]:
#Шаг 4. Мутация (инвертирование всех битов у 1 особи )
def mutation(setPair):
    randnum = random.randint(0,1)
    return [(~np.array(setPair[randnum]).astype('bool')).astype('int').tolist(),setPair[1-randnum]]

In [9]:
def newPop(setsPairs):
    columnNames = ['id', 'set', 'fitness']
    pairsCount = setsPairs["pair"].max()
    newDF = pd.DataFrame(columns=columnNames)
    for pairIDX in range(1,pairsCount+1):
        setPair = setsPairs[setsPairs['pair'] == pairIDX]['set'].values.tolist()
        children = [breed(setPair),breed(setPair)]
        children = mutation(children)
        childIDS = setsPairs[setsPairs['pair'] == pairIDX]['id'].values.tolist()
        childID = 0
        for child in children:
            childDF = pd.DataFrame([[childIDS[childID],child,fitness(child)]],columns=columnNames)
            childID = childID+1
            newDF = newDF.append(childDF)
    return newDF

In [221]:
iterationCount=0
generationSize = 200
setsDF = generation(len(priceList), generationSize)
stopReason = ''
while(True):
    prevmaxfit=setsDF['fitness'].max()    
    setsPairs = createPairs(setsDF)
    if (len(setsPairs[setsPairs['pair']!=0].values.tolist()) == 0):
        continue
    newSets = newPop(setsPairs[setsPairs['pair']!=0])
    newIDS = newSets['id'].values.tolist()
    setsDF = setsDF[~setsDF['id'].isin(newIDS)].append(newSets)
    iterationCount= iterationCount+1
    if(abs(setsDF['fitness'].max()-prevmaxfit)<min(priceList)):
        stopReason = 'Нет изменений'
        break
    if(len(setsDF[setsDF['fitness']!=0].values.tolist())<2):
        stopReason = 'Нет пар'
        break
    if(iterationCount==500):
        stopReason = 'Прошло максимальное кол-во итераций'
        break
bestSet = setsDF[setsDF['fitness']!=0].sort_values(by=['fitness'],ascending=False).iloc[0]
resSet = bestSet['set']
totalWeight = sum([a * b for a, b in zip(weightList, resSet)])
totalVolume = sum([a * b for a, b in zip(volumeList, resSet)])

print('Полученный набор: ',resSet)
print('Полученная цена: ',bestSet['fitness'])
print('Полученный вес: ',totalWeight)
print('Полученный объем: ',totalVolume)
print('Поколений: ',iterationCount+1)
print('Причина окончания: ',stopReason)

Полученный набор:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
Полученная цена:  3686
Полученный вес:  12678
Полученный объем:  11.7
Поколений:  2
Причина окончания:  Нет изменений
